In [1]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [2]:
import os 
os.chdir('drive/AI/HousePrice')

In [3]:
!pip install fastai

In [0]:
!pip install scipy==1.0.0

In [0]:
!pip install pillow==4.1.1

In [4]:
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import xgboost as xgb

In [5]:
df_raw = pd.read_csv("train.csv", low_memory = False, parse_dates=["YearBuilt","YearRemodAdd"])

In [0]:
def display_all(df):
  with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
    display(df)

In [0]:
display_all(df_raw.tail().T)

,1455,1456,1457,1458,1459
Id,1456,1457,1458,1459,1460
MSSubClass,60,20,70,20,20
MSZoning,RL,RL,RL,RL,RL
LotFrontage,62,85,66,68,75
LotArea,7917,13175,9042,9717,9937
Street,Pave,Pave,Pave,Pave,Pave
Alley,NaN,NaN,NaN,NaN,NaN
LotShape,Reg,Reg,Reg,Reg,Reg
LandContour,Lvl,Lvl,Lvl,Lvl,Lvl
Utilities,AllPub,AllPub,AllPub,AllPub,AllPub


In [6]:
add_datepart(df_raw, 'YearBuilt')
add_datepart(df_raw,'YearRemodAdd')

In [0]:
display_all(df_raw.tail()
           )

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,YearBuiltYear,YearBuiltMonth,YearBuiltWeek,YearBuiltDay,YearBuiltDayofweek,YearBuiltDayofyear,YearBuiltIs_month_end,YearBuiltIs_month_start,YearBuiltIs_quarter_end,YearBuiltIs_quarter_start,YearBuiltIs_year_end,YearBuiltIs_year_start,YearBuiltElapsed,YearRemodAddYear,YearRemodAddMonth,YearRemodAddWeek,YearRemodAddDay,YearRemodAddDayofweek,YearRemodAddDayofyear,YearRemodAddIs_month_end,YearRemodAddIs_month_start,YearRemodAddIs_quarter_end,YearRemodAddIs_quarter_start,YearRemodAddIs_year_end,YearRemodAddIs_year_start,YearRemodAddElapsed
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000,1999,1,53,1,4,1,False,True,False,True,False,True,915148800,2000,1,52,1,5,1,False,True,False,True,False,True,946684800
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000,1978,1,52,1,6,1,False,True,False,True,False,True,252460800,1988,1,53,1,4,1,False,True,False,True,False,True,567993600
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500,1941,1,1,1,2,1,False,True,False,True,False,True,-915148800,2006,1,52,1,6,1,False,True,False,True,False,True,1136073600
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,Y,FuseA,1078,0,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,NaN,Attchd,1950.0,Unf,1,240,TA,TA,Y,366,0,112,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125,1950,1,52,1,6,1,False,True,False,True,False,True,-631152000,1996,1,1,1,0,1,False,True,False,True,False,True,820454400
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,5,6,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,CBlock,TA,TA,No,BLQ,830,LwQ,290,136,1256,GasA,Gd,Y,SBrkr,1256,0,0,1256,1,0,1,1,3,1,TA,6,Typ,0,NaN,Attchd,1965.0,Fin,1,276,TA,TA,Y,736,68,0,0,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500,1965,1,53,1,4,1,False,True,False,True,False,True,-157766400,1965,1,53,1,4,1,False,True,False,True,False,True,-157766400


In [7]:
train_cats(df_raw)

In [9]:
df_raw.shape

(1460, 105)

In [0]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather("tmp/HousePrice-raw")

In [8]:
df , y, nas, mapper = proc_df(df_raw,'SalePrice', do_scale= True)

In [0]:
display_all(df.tail())

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,YearBuiltYear,YearBuiltMonth,YearBuiltWeek,YearBuiltDay,YearBuiltDayofweek,YearBuiltDayofyear,YearBuiltIs_month_end,YearBuiltIs_month_start,YearBuiltIs_quarter_end,YearBuiltIs_quarter_start,YearBuiltIs_year_end,YearBuiltIs_year_start,YearBuiltElapsed,YearRemodAddYear,YearRemodAddMonth,YearRemodAddWeek,YearRemodAddDay,YearRemodAddDayofweek,YearRemodAddDayofyear,YearRemodAddIs_month_end,YearRemodAddIs_month_start,YearRemodAddIs_quarter_end,YearRemodAddIs_quarter_start,YearRemodAddIs_year_end,YearRemodAddIs_year_start,YearRemodAddElapsed,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
1455,1.721374,0.073375,4,-0.357114,-0.260560,2,0,4,4,1,5,1,9,3,3,1,6,-0.071836,-0.517200,2,2,13,14,3,-0.570750,4,5,3,3,4,4,6,-0.973018,6,-0.288653,0.873321,-0.238122,2,1,2,5,-0.542435,0.795198,-0.120242,0.250402,-0.819964,-0.241061,0.789741,1.227585,0.163779,-0.211454,4,0.296763,7,0.600495,5,2,0.850854,2,0.311725,-0.060731,5,5,3,-0.752176,-0.100558,-0.359325,-0.116339,-0.270208,-0.068692,0,0,0,-0.087688,0.620891,-0.614439,9,5,0.918511,0.0,1.162409,0.0,0.446129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918501,0.733308,0.0,0.990242,0.0,0.735452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.733260,-0.464385,-0.074227,-0.24236
1456,1.723747,-0.872563,4,0.687385,0.266407,2,0,4,4,1,5,1,15,3,3,1,3,-0.071836,0.381743,2,2,10,11,4,0.087911,4,5,2,3,4,4,1,0.759659,5,0.722112,0.049262,1.104925,2,5,2,5,2.355701,-0.795163,-0.120242,1.061367,1.107810,-0.241061,0.789741,-0.761621,0.163779,-0.211454,4,0.296763,3,2.152216,5,2,-0.024555,3,0.311725,0.126420,5,5,3,2.033231,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,0,3,0,-0.087688,-1.599111,1.645210,9,5,0.222975,0.0,1.123209,0.0,1.459112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.222986,0.151865,0.0,1.029211,0.0,0.269006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151816,-0.464385,-0.074227,-0.24236
1457,1.726120,0.309859,4,-0.175462,-0.147810,2,0,4,4,1,5,1,7,3,3,1,6,0.651479,3.078570,2,2,6,6,3,-0.570750,1,3,5,4,2,4,3,-0.369871,6,-0.288653,0.701265,0.215641,2,1,2,5,0.065656,1.844744,-0.120242,1.569647,-0.819964,-0.241061,0.789741,-0.761621,1.390023,-0.211454,3,1.527656,7,2.152216,3,2,-1.566941,2,-1.026858,-1.033914,5,5,3,-0.752176,0.201405,-0.359325,-0.116339,-0.270208,-0.068692,0,1,3,4.953112,-0.489110,1.645210,9,5,-1.002492,0.0,-0.876029,0.0,-0.566854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.002461,1.024029,0.0,0.990242,0.0,1.201899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.024049,-0.464385,-0.074227,-0.24236
1458,1.728492,-0.872563,4,-0.084636,-0.080160,2,0,4,4,1,5,1,13,3,3,1,3,-0.795151,0.381743,4,2,9,9,3,-0.570750,4,5,2,4,4,3,3,-0.865548,5,6.092188,-1.284176,0.046905,2,3,2,1,-0.218982,-0.795163,-0.120242,-0.832788,1.107810,-0.241061,-1.026041,-0.761621,-1.062465,-0.211454,3,-0.934130,7,-0.951226,0,2,-1.191766,3,-1.026858,-1.090059,5,5,3,2.168910,-0.704483,1.473789,-0.116339,-0.270208,-0.068692,0,0,0,-0.087688,-0.859110,1.645210,9,5,-0.704406,0.0,1.123209,0.0,1.459112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.704397,0.539493,0.0,-0.997182,0.0,-1.596781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.539446,-0.464385,-0.074227,-0.24236
1459,1.730865,-0.872563,4,0.233255,-0.058112,2,0,4,4,1,5,1,8,3,3,1,3,-0.795151,0.381743,2,2,7,7,3,-0.570750,3,5,2,4,4,4,2,0.847389,4,1.509640,-0.976285,0.452784,2,3,2,5,0.241615,-

In [10]:
model = xgb.XGBClassifier(max_depth= 50 ,min_child_weight=1, n_estimators= 60,  n_jobs=-1, verbose =1, learning_rate =0.16)
model.fit(df,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.16, max_delta_step=0,
       max_depth=50, min_child_weight=1, missing=None, n_estimators=60,
       n_jobs=-1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1, verbose=1)

In [0]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
  res = [rmse(m.predict(df),y), m.score(df,y)]
  if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
  print(res)
  

**r^2 does not improves on changing the min_samples_leaf**

In [0]:
%time preds = np.stack([t.predict(df) for t in m.estimators_] )
np.mean(preds[:,0]), np.std(preds[:,0])

CPU times: user 119 ms, sys: 2 ms, total: 121 ms
Wall time: 125 ms


(206848.3125, 9841.415499172044)

In [0]:
def get_preds(t): return t.predict(df)
%time preds = np.stack(parallel_trees(m, get_preds))
np.mean(preds[:,0]), np.std(preds[:,0])

CPU times: user 128 ms, sys: 71 ms, total: 199 ms
Wall time: 349 ms


(206848.3125, 9841.415499172044)

In [11]:
df_test = pd.read_csv("test.csv", low_memory= False , parse_dates= ['YearBuilt','YearRemodAdd'])

In [0]:
display_all(df_test.tail())

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970-01-01,1970-01-01,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970-01-01,1970-01-01,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960-01-01,1996-01-01,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992-01-01,1992-01-01,Gable,CompShg,HdBoard,Wd Shng,None,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,337.0,Unf,0.0,575.0,912.0,GasA,TA,Y,SBrkr,970,0,0,970,0.0,1.0,1,0,3,1,TA,6,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,80,32,0,0,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal
1458,2919,60,RL,74.0,9627,Pave,NaN,Reg,Lvl,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,2Story,7,5,1993-01-01,1994-01-01,Gable,CompShg,HdBoard,HdBoard,BrkFace,94.0,TA,TA,PConc,Gd,TA,Av,LwQ,758.0,Unf,0.0,238.0,996.0,GasA,Ex,Y,SBrkr,996,1004,0,2000,0.0,0.0,2,1,3,1,TA,9,Typ,1,TA,Attchd,1993.0,Fin,3.0,650.0,TA,TA,Y,190,48,0,0,0,0,NaN,NaN,NaN,0,11,2006,WD,Normal


In [12]:
add_datepart(df_test,'YearBuilt')
add_datepart(df_test,'YearRemodAdd')


In [13]:
df_test.shape

(1459, 104)

In [14]:
train_cats(df_test)

In [15]:
df_test_proc, _, nas, mapper = proc_df(df_test, y_fld=None, do_scale=True, mapper=mapper, na_dict=nas)


In [0]:
display_all(df_test_proc.tail())

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,YearBuiltYear,YearBuiltMonth,YearBuiltWeek,YearBuiltDay,YearBuiltDayofweek,YearBuiltDayofyear,YearBuiltIs_month_end,YearBuiltIs_month_start,YearBuiltIs_quarter_end,YearBuiltIs_quarter_start,YearBuiltIs_year_end,YearBuiltIs_year_start,YearBuiltElapsed,YearRemodAddYear,YearRemodAddMonth,YearRemodAddWeek,YearRemodAddDay,YearRemodAddDayofweek,YearRemodAddDayofyear,YearRemodAddIs_month_end,YearRemodAddIs_month_start,YearRemodAddIs_quarter_end,YearRemodAddIs_quarter_start,YearRemodAddIs_year_end,YearRemodAddIs_year_start,YearRemodAddElapsed,LotFrontage_na,MasVnrArea_na,BsmtFinSF1_na,BsmtFinSF2_na,BsmtUnfSF_na,TotalBsmtSF_na,BsmtFullBath_na,BsmtHalfBath_na,GarageYrBlt_na,GarageCars_na,GarageArea_na
1454,5.183104,2.438219,5,-2.219047,-0.859988,2,0,4,4,1,5,1,11,3,3,4,5,-1.518467,1.280685,2,1,6,6,3,-0.570750,4,5,2,4,4,4,6,-0.973018,6,-0.288653,-0.048086,-1.166169,1,3,2,4,-1.595596,0.456043,-0.120242,-0.806136,-0.819964,-0.241061,-1.026041,1.227585,0.163779,-0.211454,4,-0.934130,7,-0.951226,0,0,0.058817,0,-2.365440,-2.212963,0,0,3,-0.752176,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,0,0,0,-0.087688,-0.119110,-1.367655,9,5,-0.041991,0.0,-0.876029,0.0,-0.060363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.041980,-0.720298,0.0,-0.997182,0.0,-0.197441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.720285,-0.464385,-0.074227,False,False,False,False,False,False,4.126099,False,False
1455,5.185477,2.438219,5,-2.219047,-0.864197,2,0,4,4,1,5,1,11,3,3,5,5,-1.518467,-0.517200,2,1,6,6,3,-0.570750,4,5,2,4,4,4,5,-0.420316,6,-0.288653,-0.618589,-1.166169,1,5,2,4,-1.595596,0.456043,-0.120242,-0.806136,-0.819964,-0.241061,-1.026041,1.227585,0.163779,-0.211454,4,-0.318683,7,-0.951226,0,5,-0.358044,3,-1.026858,-0.874836,4,5,3,-0.752176,-0.342128,-0.359325,-0.116339,-0.270208,-0.068692,0,0,0,-0.087688,-0.859110,-1.367655,9,1,-0.041991,0.0,-0.876029,0.0,-0.060363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.041980,-0.720298,0.0,-0.997182,0.0,-0.197441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.720285,-0.464385,-0.074227,False,False,False,False,False,False,-0.242360,False,False
1456,5.187849,-0.872563,4,4.093359,0.950423,2,0,4,4,1,5,1,12,3,3,1,3,-0.795151,1.280685,2,1,11,13,3,-0.570750,4,5,2,4,4,4,1,1.711535,6,-0.288653,-1.284176,0.379817,1,1,2,4,0.158811,-0.795163,-0.120242,-0.554851,1.107810,-0.241061,-1.026041,-0.761621,1.390023,-0.211454,4,0.296763,7,0.600495,5,6,-0.774905,3,0.311725,0.482006,4,5,3,3.030870,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,0,0,0,-0.087688,0.990891,-1.367655,9,1,-0.373198,0.0,1.162409,0.0,0.446129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.373234,0.539493,0.0,-0.997182,0.0,-1.596781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.539446,-0.464385,-0.074227,False,False,False,False,False,False,-0.242360,False,False
1457,5.190222,0.664586,4,-0.357114,-0.007600,2,0,4,4,1,5,1,12,3,3,1,6,-0.795151,-0.517200,2,1,7,15,3,-0.570750,4,5,3,3,4,1,3,-0.233889,6,-0.288653,0.017567,-0.331610,1,5,2,4,-0.498445,-0.795163,-0.120242,-1.038384,-0.819964,3.948809,-1.026041,-0.761621,0.163779,-0.211454,4,-0.318683,7,-0.951226,0,0,0.058817,0,-2.365440,-2.212963,0,0,3,-0.113687,-0.221343,-0.359325,-0.116339,-0.270208,-0.068692,0,3,3,1.323736,0.250891,-1.367655,9,5,0.686666,0.0,-0.876029,0.0,-0.566854,0.0,0.0,0.0,0.0,0.

In [0]:
display_all(df.tail())

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,YearBuiltYear,YearBuiltMonth,YearBuiltWeek,YearBuiltDay,YearBuiltDayofweek,YearBuiltDayofyear,YearBuiltIs_month_end,YearBuiltIs_month_start,YearBuiltIs_quarter_end,YearBuiltIs_quarter_start,YearBuiltIs_year_end,YearBuiltIs_year_start,YearBuiltElapsed,YearRemodAddYear,YearRemodAddMonth,YearRemodAddWeek,YearRemodAddDay,YearRemodAddDayofweek,YearRemodAddDayofyear,YearRemodAddIs_month_end,YearRemodAddIs_month_start,YearRemodAddIs_quarter_end,YearRemodAddIs_quarter_start,YearRemodAddIs_year_end,YearRemodAddIs_year_start,YearRemodAddElapsed,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
1455,1.721374,0.073375,4,-0.357114,-0.260560,2,0,4,4,1,5,1,9,3,3,1,6,-0.071836,-0.517200,2,2,13,14,3,-0.570750,4,5,3,3,4,4,6,-0.973018,6,-0.288653,0.873321,-0.238122,2,1,2,5,-0.542435,0.795198,-0.120242,0.250402,-0.819964,-0.241061,0.789741,1.227585,0.163779,-0.211454,4,0.296763,7,0.600495,5,2,0.850854,2,0.311725,-0.060731,5,5,3,-0.752176,-0.100558,-0.359325,-0.116339,-0.270208,-0.068692,0,0,0,-0.087688,0.620891,-0.614439,9,5,0.918511,0.0,1.162409,0.0,0.446129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918501,0.733308,0.0,0.990242,0.0,0.735452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.733260,-0.464385,-0.074227,-0.24236
1456,1.723747,-0.872563,4,0.687385,0.266407,2,0,4,4,1,5,1,15,3,3,1,3,-0.071836,0.381743,2,2,10,11,4,0.087911,4,5,2,3,4,4,1,0.759659,5,0.722112,0.049262,1.104925,2,5,2,5,2.355701,-0.795163,-0.120242,1.061367,1.107810,-0.241061,0.789741,-0.761621,0.163779,-0.211454,4,0.296763,3,2.152216,5,2,-0.024555,3,0.311725,0.126420,5,5,3,2.033231,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,0,3,0,-0.087688,-1.599111,1.645210,9,5,0.222975,0.0,1.123209,0.0,1.459112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.222986,0.151865,0.0,1.029211,0.0,0.269006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151816,-0.464385,-0.074227,-0.24236
1457,1.726120,0.309859,4,-0.175462,-0.147810,2,0,4,4,1,5,1,7,3,3,1,6,0.651479,3.078570,2,2,6,6,3,-0.570750,1,3,5,4,2,4,3,-0.369871,6,-0.288653,0.701265,0.215641,2,1,2,5,0.065656,1.844744,-0.120242,1.569647,-0.819964,-0.241061,0.789741,-0.761621,1.390023,-0.211454,3,1.527656,7,2.152216,3,2,-1.566941,2,-1.026858,-1.033914,5,5,3,-0.752176,0.201405,-0.359325,-0.116339,-0.270208,-0.068692,0,1,3,4.953112,-0.489110,1.645210,9,5,-1.002492,0.0,-0.876029,0.0,-0.566854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.002461,1.024029,0.0,0.990242,0.0,1.201899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.024049,-0.464385,-0.074227,-0.24236
1458,1.728492,-0.872563,4,-0.084636,-0.080160,2,0,4,4,1,5,1,13,3,3,1,3,-0.795151,0.381743,4,2,9,9,3,-0.570750,4,5,2,4,4,3,3,-0.865548,5,6.092188,-1.284176,0.046905,2,3,2,1,-0.218982,-0.795163,-0.120242,-0.832788,1.107810,-0.241061,-1.026041,-0.761621,-1.062465,-0.211454,3,-0.934130,7,-0.951226,0,2,-1.191766,3,-1.026858,-1.090059,5,5,3,2.168910,-0.704483,1.473789,-0.116339,-0.270208,-0.068692,0,0,0,-0.087688,-0.859110,1.645210,9,5,-0.704406,0.0,1.123209,0.0,1.459112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.704397,0.539493,0.0,-0.997182,0.0,-1.596781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.539446,-0.464385,-0.074227,-0.24236
1459,1.730865,-0.872563,4,0.233255,-0.058112,2,0,4,4,1,5,1,8,3,3,1,3,-0.795151,0.381743,2,2,7,7,3,-0.570750,3,5,2,4,4,4,2,0.847389,4,1.509640,-0.976285,0.452784,2,3,2,5,0.241615,-

In [16]:
df_test_proc.drop(['BsmtFinSF1_na','BsmtFinSF2_na','BsmtFullBath_na','BsmtHalfBath_na','BsmtUnfSF_na','GarageArea_na','GarageCars_na','TotalBsmtSF_na'] , axis= 1, inplace= True)

In [17]:
predicted_price = model.predict(df_test_proc)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
my_submission= pd.DataFrame({'Id': df_test.Id, 'SalePrice' : predicted_price})
my_submission.to_csv('XGboost1.csv', index = False)